In [5]:
import pandas as pd
import numpy as np
import os
import datarobot as dr
import time
from pprint import pprint

USERNAME = os.environ['DATAROBOT_USERNAME']
API_TOKEN = os.environ['DATAROBOT_API_TOKEN']
ENDPOINT = 'https://app.datarobot.com/api/v2'

# USERNAME = 'matthew.cohen=demo@datarobot.com'
# API_TOKEN = 'NWQ1NDA3OGRmNTU1Y2QxZDQxNmJlZmQ2OjZxR3RCbWxudll6cTFWeFBuRS1lN3dhWVBIN1hYLU42'

dr.Client(token=API_TOKEN, endpoint=ENDPOINT) 

In [6]:
TRAIN = df = pd.read_csv('/Users/matthew.cohen/Documents/DR/Customers/Veem/data/df_train.csv')
TEST = df = pd.read_csv('/Users/matthew.cohen/Documents/DR/Customers/Veem/data/df_test.csv')

PROJECT_ID = '616f0ec1892810e8a6c70adc'
prj = dr.Project.get(PROJECT_ID)

print('Project name and file: %s, %s' % (prj.project_name, prj.file_name))

models = prj.get_models()  # [:12]  # get only the top n to narrow it down
print('%s models' % len(models))

Project name and file: Store_Sales_Forecast_train.csv, Store_Sales_Forecast_train.csv
30 models


In [7]:
# # Get a model with coefficients
# for i, m in enumerate(models):
#     if "Binned" in m.model_type:
#         m = models[i]
#         break
# params = m.get_parameters()
# params.derived_features

## Run Feature Impact and Feature Effects.  
### These calls return program flow as they are threaded and run asynchronously.

In [8]:
# m = models[0]
print('Getting feature impact')
for i, m in enumerate(models):
    print('%s. %s' % (i+1, m.model_type))
    try:
        m.request_feature_impact()
    except dr.errors.JobAlreadyRequested as jar:
#         import pdb; pdb.set_trace()
        print('- Feature Impact already requested')
    except Exception as e:
        print('- Feature Impact error:', e)
    
print()
    
print('Getting feature effects')
for i, m in enumerate(models):
    print('%s. %s' % (i+1, m.model_type))
    try:
        if prj.is_datetime_partitioned == True:
            # cast as a datetime model
            m = dr.DatetimeModel.get(prj.id, m.id)
            m.request_feature_effect(backtest_index='0')
        else:
#             m.request_feature_effect(data_subset=dr.enums.DATA_SUBSET.ALL)
            m.request_feature_effect()
    except dr.errors.JobAlreadyRequested as jar:
        print('- Feature Effects already requested')
    except Exception as e:
        print('- Feature Effects error:', e)

Getting feature impact
1. eXtreme Gradient Boosted Trees Regressor
- Feature Impact already requested
2. AVG Blender
3. eXtreme Gradient Boosted Trees Regressor
4. eXtreme Gradient Boosted Trees Regressor
- Feature Impact already requested
5. Zero-Inflated Light Gradient Boosted Trees Regressor with Early Stopping (Gamma Loss)
6. Zero-Inflated eXtreme Gradient Boosted Trees Regressor (Poisson Loss)
7. Temporal Hierarchical Model with Elastic Net and XGBoost
8. Elastic-Net Regressor (mixing alpha=0.5 / Least-Squares Loss) with Forecast Distance Modeling
9. Ridge Regressor with Forecast Distance Modeling
10. eXtreme Gradient Boosted Trees Regressor
11. Keras DeepAR LSTM Regressor with Student's T Probabilistic Distribution Modeling using Training Schedule (1 LSTM Layer: 40 Units)
12. Temporal Hierarchical Model with Elastic Net and XGBoost
13. Ridge Regressor with Forecast Distance Modeling
14. eXtreme Gradient Boosted Trees Regressor
15. Eureqa Generalized Additive Model (250 Generation

## Run Prediction Explanations.  Feature Impact must complete first.
### These calls return program flow as they are threaded and run asynchronously.  

In [11]:
print('Getting Prediction Explanations')
for i, m in enumerate(models):
    print('%s. %s' % (i+1, m.model_type))
    try:
        pei_job = dr.PredictionExplanationsInitialization.create(prj.id, m.id)
    except Exception as e:
        pass  

# pei_job.wait_for_completion()

Getting Prediction Explanations
1. eXtreme Gradient Boosted Trees Classifier with Early Stopping
2. ENET Blender
3. AVG Blender
4. ENET Blender
5. Gradient Boosted Trees Classifier with Early Stopping
6. Advanced AVG Blender
7. Light Gradient Boosted Trees Classifier with Early Stopping
8. Light Gradient Boosted Trees Classifier with Early Stopping
9. Light Gradient Boosted Trees Classifier with Early Stopping
10. eXtreme Gradient Boosted Trees Classifier with Early Stopping
11. Generalized Additive2 Model
12. Light Gradient Boosted Trees Classifier with Early Stopping
13. Gradient Boosted Greedy Trees Classifier with Early Stopping
14. Light Gradient Boosted Trees Classifier with Early Stopping
15. Gradient Boosted Trees Classifier with Early Stopping
16. Gradient Boosted Greedy Trees Classifier with Early Stopping
17. RandomForest Classifier (Entropy)
18. Generalized Additive2 Model
19. Elastic-Net Classifier (mixing alpha=0.5 / Binomial Deviance)
20. eXtreme Gradient Boosted Trees C

## Request external test

In [43]:
# ext_test.id
# dr.ExternalScores.list(prj.id, dataset_id=ext_test.id)

ext_test = prj.upload_dataset(TEST)
print('external dataset: %s, id: %s' % (ext_test, ext_test.id))

KeyboardInterrupt: 

## For permuting columns

In [110]:
# Upload dataset
# dataset = prj.upload_dataset('./test_set.csv')
# print('dataset.contains_target_values:', dataset.contains_target_values)

import pandas as pd
import numpy as np
import os
import datarobot as dr
import time
from pprint import pprint

USERNAME = os.environ['DATAROBOT_USERNAME']
API_TOKEN = os.environ['DATAROBOT_API_TOKEN']
ENDPOINT = 'https://app.datarobot.com/api/v2'

dr.Client(token=API_TOKEN, endpoint=ENDPOINT) 

t0 = time.time()
prj = dr.Project.get(PROJECT_ID) # 10k_diabetes_80
print('project: ', prj)

df = pd.read_excel('/Users/matthew.cohen/Documents/DR/Demo/_Diabetes/10k_diabetes_20.xlsx')

num_prj_models_to_run = 3

for i, col in enumerate(df.columns):
    print('\n**** permuting column \'%s\' (%s of %s) ****\n' % (col, i+1, len(df.columns)))
    df[col] = np.random.permutation(df[col])
    
    # Upload the external dataset
    ext_test = prj.upload_dataset(df)
    print('external dataset: %s, id: %s' % (ext_test, ext_test.id))

    # Request n models request/run the external dataset
    print('Request external dataset test for %s models.' % num_prj_models_to_run)
    for i, model in enumerate(models):
        if i+1 > num_prj_models_to_run:
            break
        model = models[i]
        # print('request model %s %s for external test set' % (i, model))
        model.request_external_test(ext_test.id)

    # Wait for each model to complete scoring the external test set scores before getting the scores
    while len(prj.get_all_jobs()) > 0:
        time.sleep(1)
    print('- Externals test requests complete.')

    # Get the scores for the external dataset for each model
    for i, score in enumerate(dr.ExternalScores.list(prj.id, dataset_id=ext_test.id)):
        cur_model = dr.models.Model.get(project=prj.id, model_id=score.model_id)
        print('model %s: %s, id: %s' % (i, 
                                        cur_model.model_type,
                                        score.model_id)
             )
        pprint(score.scores)
        
        while len(prj.get_all_jobs()) > 0:
            time.sleep(1)
    
    # Delete the external dataset
    print('\ndeleting dataset:', ext_test.id)
    ext_test.delete()
 
print('Time: %d.03 minutes' % ((time.time() - t0)/60))

project:  Project(10k_diabetes_80.xlsx)

**** permuting column 'race' (1 of 51) ****

external dataset: PredictionDataset('predict.csv'), id: 60a5f6018ae14ab4a51697d7
Request external dataset test for 3 models.
- Externals test requests complete.
model 0: Light Gradient Boosted Trees Classifier with Early Stopping, id: 602d6671406e5972767bd275
[{'label': 'AUC', 'value': 0.7138},
 {'label': 'Area Under PR Curve', 'value': 0.61071},
 {'label': 'FVE Binomial', 'value': 0.10534},
 {'label': 'Gini Norm', 'value': 0.4276},
 {'label': 'Kolmogorov-Smirnov', 'value': 0.32514},
 {'label': 'LogLoss', 'value': 0.60082},
 {'label': 'Max MCC', 'value': 0.31822},
 {'label': 'RMSE', 'value': 0.45553},
 {'label': 'Rate@Top10%', 'value': 0.74},
 {'label': 'Rate@Top5%', 'value': 0.76},
 {'label': 'Rate@TopTenth%', 'value': 0.5}]
model 1: Light Gradient Boosted Trees Classifier with Early Stopping, id: 602d6718bc10c707f16bc0c1
[{'label': 'AUC', 'value': 0.71589},
 {'label': 'Area Under PR Curve', 'value':

model 1: Light Gradient Boosted Trees Classifier with Early Stopping, id: 602d6671406e5972767bd275
[{'label': 'AUC', 'value': 0.69689},
 {'label': 'Area Under PR Curve', 'value': 0.58423},
 {'label': 'FVE Binomial', 'value': 0.09012},
 {'label': 'Gini Norm', 'value': 0.39378},
 {'label': 'Kolmogorov-Smirnov', 'value': 0.28977},
 {'label': 'LogLoss', 'value': 0.61104},
 {'label': 'Max MCC', 'value': 0.28385},
 {'label': 'RMSE', 'value': 0.4606},
 {'label': 'Rate@Top10%', 'value': 0.71},
 {'label': 'Rate@Top5%', 'value': 0.73},
 {'label': 'Rate@TopTenth%', 'value': 0.5}]
model 2: Light Gradient Boosted Trees Classifier with Early Stopping, id: 602d64db1be244c02d7bd243
[{'label': 'AUC', 'value': 0.69692},
 {'label': 'Area Under PR Curve', 'value': 0.58407},
 {'label': 'FVE Binomial', 'value': 0.08643},
 {'label': 'Gini Norm', 'value': 0.39384},
 {'label': 'Kolmogorov-Smirnov', 'value': 0.30358},
 {'label': 'LogLoss', 'value': 0.61352},
 {'label': 'Max MCC', 'value': 0.2991},
 {'label': 'R

model 2: Light Gradient Boosted Trees Classifier with Early Stopping, id: 602d64db1be244c02d7bd243
[{'label': 'AUC', 'value': 0.64196},
 {'label': 'Area Under PR Curve', 'value': 0.53701},
 {'label': 'FVE Binomial', 'value': 0.03954},
 {'label': 'Gini Norm', 'value': 0.28392},
 {'label': 'Kolmogorov-Smirnov', 'value': 0.22113},
 {'label': 'LogLoss', 'value': 0.64501},
 {'label': 'Max MCC', 'value': 0.23227},
 {'label': 'RMSE', 'value': 0.47508},
 {'label': 'Rate@Top10%', 'value': 0.605},
 {'label': 'Rate@Top5%', 'value': 0.67},
 {'label': 'Rate@TopTenth%', 'value': 0.5}]

deleting dataset: 60a5f86d8ae14ab4a5169836

**** permuting column 'medical_specialty' (10 of 51) ****

external dataset: PredictionDataset('predict.csv'), id: 60a5f8bc570fec41a11697ec
Request external dataset test for 3 models.
- Externals test requests complete.
model 0: Light Gradient Boosted Trees Classifier with Early Stopping, id: 602d6718bc10c707f16bc0c1
[{'label': 'AUC', 'value': 0.6447},
 {'label': 'Area Under


deleting dataset: 60a5f9974d8eb04bc1f971b0

**** permuting column 'number_outpatient' (14 of 51) ****

external dataset: PredictionDataset('predict.csv'), id: 60a5f9dcc31d97ad10169717
Request external dataset test for 3 models.
- Externals test requests complete.
model 0: Light Gradient Boosted Trees Classifier with Early Stopping, id: 602d6718bc10c707f16bc0c1
[{'label': 'AUC', 'value': 0.62202},
 {'label': 'Area Under PR Curve', 'value': 0.51593},
 {'label': 'FVE Binomial', 'value': 0.0205},
 {'label': 'Gini Norm', 'value': 0.24404},
 {'label': 'Kolmogorov-Smirnov', 'value': 0.20177},
 {'label': 'LogLoss', 'value': 0.6578},
 {'label': 'Max MCC', 'value': 0.19832},
 {'label': 'RMSE', 'value': 0.48029},
 {'label': 'Rate@Top10%', 'value': 0.63},
 {'label': 'Rate@Top5%', 'value': 0.64},
 {'label': 'Rate@TopTenth%', 'value': 0.5}]
model 1: Light Gradient Boosted Trees Classifier with Early Stopping, id: 602d6671406e5972767bd275
[{'label': 'AUC', 'value': 0.62041},
 {'label': 'Area Under P

model 0: Light Gradient Boosted Trees Classifier with Early Stopping, id: 602d6671406e5972767bd275
[{'label': 'AUC', 'value': 0.55102},
 {'label': 'Area Under PR Curve', 'value': 0.43555},
 {'label': 'FVE Binomial', 'value': -0.03165},
 {'label': 'Gini Norm', 'value': 0.10204},
 {'label': 'Kolmogorov-Smirnov', 'value': 0.09272},
 {'label': 'LogLoss', 'value': 0.69283},
 {'label': 'Max MCC', 'value': 0.09369},
 {'label': 'RMSE', 'value': 0.49678},
 {'label': 'Rate@Top10%', 'value': 0.46},
 {'label': 'Rate@Top5%', 'value': 0.5},
 {'label': 'Rate@TopTenth%', 'value': 0.5}]
model 1: Light Gradient Boosted Trees Classifier with Early Stopping, id: 602d6718bc10c707f16bc0c1
[{'label': 'AUC', 'value': 0.56338},
 {'label': 'Area Under PR Curve', 'value': 0.44603},
 {'label': 'FVE Binomial', 'value': -0.02649},
 {'label': 'Gini Norm', 'value': 0.12676},
 {'label': 'Kolmogorov-Smirnov', 'value': 0.11188},
 {'label': 'LogLoss', 'value': 0.68935},
 {'label': 'Max MCC', 'value': 0.11033},
 {'label':

model 1: Light Gradient Boosted Trees Classifier with Early Stopping, id: 602d64db1be244c02d7bd243
[{'label': 'AUC', 'value': 0.53625},
 {'label': 'Area Under PR Curve', 'value': 0.43051},
 {'label': 'FVE Binomial', 'value': -0.02562},
 {'label': 'Gini Norm', 'value': 0.0725},
 {'label': 'Kolmogorov-Smirnov', 'value': 0.06958},
 {'label': 'LogLoss', 'value': 0.68878},
 {'label': 'Max MCC', 'value': 0.06852},
 {'label': 'RMSE', 'value': 0.49581},
 {'label': 'Rate@Top10%', 'value': 0.47},
 {'label': 'Rate@Top5%', 'value': 0.45},
 {'label': 'Rate@TopTenth%', 'value': 0.5}]
model 2: Light Gradient Boosted Trees Classifier with Early Stopping, id: 602d6671406e5972767bd275
[{'label': 'AUC', 'value': 0.53852},
 {'label': 'Area Under PR Curve', 'value': 0.4218},
 {'label': 'FVE Binomial', 'value': -0.03987},
 {'label': 'Gini Norm', 'value': 0.07704},
 {'label': 'Kolmogorov-Smirnov', 'value': 0.07267},
 {'label': 'LogLoss', 'value': 0.69834},
 {'label': 'Max MCC', 'value': 0.07736},
 {'label': 

model 2: Light Gradient Boosted Trees Classifier with Early Stopping, id: 602d64db1be244c02d7bd243
[{'label': 'AUC', 'value': 0.53639},
 {'label': 'Area Under PR Curve', 'value': 0.43162},
 {'label': 'FVE Binomial', 'value': -0.02578},
 {'label': 'Gini Norm', 'value': 0.07278},
 {'label': 'Kolmogorov-Smirnov', 'value': 0.06533},
 {'label': 'LogLoss', 'value': 0.68888},
 {'label': 'Max MCC', 'value': 0.06745},
 {'label': 'RMSE', 'value': 0.49581},
 {'label': 'Rate@Top10%', 'value': 0.47},
 {'label': 'Rate@Top5%', 'value': 0.49},
 {'label': 'Rate@TopTenth%', 'value': 0.5}]

deleting dataset: 60a60da24cdf3d9e39f971ac

**** permuting column 'glimepiride' (27 of 51) ****



ConnectionError: ('Connection aborted.', OSError("(32, 'EPIPE')"))

In [100]:
dr.models.Model.get(project = prj.id, model_id = '602d64db1be244c02d7bd243').model_type
dr.models.Model.get(project = prj.id, model_id = '602d64db1be244c02d7bd243')

'Light Gradient Boosted Trees Classifier with Early Stopping'